In [5]:
import torch
import numpy as np

from torch import nn
import torch.nn.functional as F

In [20]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [8]:
def conv3x3(in_channels, out_channels, stride=1, groups=1, dilation=-1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, groups=1, dilation=-1)

def conv1x1(in_channels, out_channels, stride=1, groups=1, dilation=-1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, groups=1, dilation=-1)

In [16]:
class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super().__init__()
        
        self.conv1 = conv3x3(in_channels=in_channels, out_channels=out_channels)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = conv3x3(in_channels=out_channels, out_channels=out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.skip = nn.Sequential()
        
    def forward(self, x):
        
        out = F.relu(self.conv1(x))
        out = self.bn1(out)
        out = F.relu(self.conv2(out))
        out = self.bn2(out)
        
        out += self.Sequential()
        
        return out

In [17]:
class ResNet(nn.Module):
    
    def __init__(self, num_classes=10):
        super().__init__()
        
        self.conv1 = conv3x3(in_channels=3, out_channels=64)
        
        self.b1 = ResidualBlock(in_channels=64, out_channels=128)
        self.b2 = ResidualBlock(in_channels=128, out_channels=256)
        self.b3 = ResidualBlock(in_channels=256, out_channels=256)
        self.b4 = ResidualBlock(in_channels=256, out_channels=512)
        
        self.linear = nn.Linear(512, 10)
    
    def forward(self, x):
        
        out = self.conv1(x)
        out = self.b1(out)
        out = self.b2(out)
        out = self.b3(out)
        out = self.b4(out)
        
        out = self.linear(out)
        
        return out
        
        

In [18]:
import torchvision.transforms as transforms
import torchvision

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [27]:
model = ResNet()
model.to(device)

loss_func = nn.CrossEntropyLoss()

opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [31]:
for data in trainset:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    
    pred = model(inputs)
    
    loss = loss_func(pred, labels)
    
    loss.backward()
    
    opt.step()
    
    opt.zero_grad()
    
    

AttributeError: 'int' object has no attribute 'to'